<a href="https://colab.research.google.com/github/marvelous-koala/unlimited_power/blob/master/LSTM_mk1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd
from google.colab import drive
from pandas import Series, DataFrame
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
from pandas import DataFrame
from pandas import concat
import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
# 자료 불러오기
raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
# "Day" 칼럼은 의미가 없으니까 제외하기
raw_data.drop('Day', inplace=True, axis = 1)

# 지금 칼럼은 시간(Hour)과 분으로 2개가 있는데, 얘를 10:30분 이렇게 합쳐준다.
# 시간 + 분 = 11.5 (11시 30분), 22.0 (22시 정각) 이런 식으로 변환
raw_data['Time'] = raw_data['Hour'] + raw_data['Minute']*(0.5/30)

# 이제 Hour랑 Minute 칼럼은 필요 없으니까 버린다.
raw_data.drop(['Hour', 'Minute'], axis=1, inplace=True)

# Target이 Dataframe의 맨 끝에 가도록 예쁘게 수정
raw_data = raw_data[['Time', 'DHI','DNI','WS','RH','T','TARGET']]
raw_data

,Time,DHI,DNI,WS,RH,T,TARGET
0,0.0,0,0,1.5,69.08,-12,0.0
1,0.5,0,0,1.5,69.06,-12,0.0
2,1.0,0,0,1.6,71.78,-12,0.0
3,1.5,0,0,1.6,71.75,-12,0.0
4,2.0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...
52555,21.5,0,0,2.4,70.70,-4,0.0
52556,22.0,0,0,2.4,66.79,-4,0.0
52557,22.5,0,0,2.2,66.78,-4,0.0
52558,23.0,0,0,2.1,67.72,-4,0.0


In [102]:
# -1에서 1까지 스케일링 하기도 하던데.. 일단 잘 모르니까 무난하게 0-1로 스케일링
scaler = MinMaxScaler()

# df_scaled = pd.DataFrame(scaler.fit_transform(raw_data), columns=raw_data.columns, index = raw_data.index)
df_scaled = raw_data.copy()
df_scaled.head(5)

,Time,DHI,DNI,WS,RH,T,TARGET
0,0.0,0,0,1.5,69.08,-12,0.0
1,0.5,0,0,1.5,69.06,-12,0.0
2,1.0,0,0,1.6,71.78,-12,0.0
3,1.5,0,0,1.6,71.75,-12,0.0
4,2.0,0,0,1.6,75.20,-12,0.0


In [103]:
# 예측 모델에 넣은 데이터의 형태를 정의
# 7일의 데이터를 학습하여 2일을 예측하는 경우
# 하루 48개 틱, 예측에 일단 7일을 전부 사용하는 경우를 가정
ticks = 48
days = 7
n_days = ticks*days

# 미래 2일을 예측
future_days = 2
future_window = ticks * future_days 

In [104]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 함수
def series_to_supervised(data, n_in=1, n_out=1, target = 'TARGET', dropnan=True):
    df = DataFrame(data)
    df.drop(target, axis = 1, inplace=True)
    df2 = DataFrame(data[target])
    cols, names = list(), list()
    n_vars = 1 if type(df) is list else df.shape[1]
    n_vars2 = 1 if type(df2) is list else df2.shape[1]
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df2.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [105]:
reframed = series_to_supervised(df_scaled, n_days, future_window)
reframed

,var1(t-336),var2(t-336),var3(t-336),var4(t-336),var5(t-336),var6(t-336),var1(t-335),var2(t-335),var3(t-335),var4(t-335),var5(t-335),var6(t-335),var1(t-334),var2(t-334),var3(t-334),var4(t-334),var5(t-334),var6(t-334),var1(t-333),var2(t-333),var3(t-333),var4(t-333),var5(t-333),var6(t-333),var1(t-332),var2(t-332),var3(t-332),var4(t-332),var5(t-332),var6(t-332),var1(t-331),var2(t-331),var3(t-331),var4(t-331),var5(t-331),var6(t-331),var1(t-330),var2(t-330),var3(t-330),var4(t-330),...,TARGET1(t+56),TARGET1(t+57),TARGET1(t+58),TARGET1(t+59),TARGET1(t+60),TARGET1(t+61),TARGET1(t+62),TARGET1(t+63),TARGET1(t+64),TARGET1(t+65),TARGET1(t+66),TARGET1(t+67),TARGET1(t+68),TARGET1(t+69),TARGET1(t+70),TARGET1(t+71),TARGET1(t+72),TARGET1(t+73),TARGET1(t+74),TARGET1(t+75),TARGET1(t+76),TARGET1(t+77),TARGET1(t+78),TARGET1(t+79),TARGET1(t+80),TARGET1(t+81),TARGET1(t+82),TARGET1(t+83),TARGET1(t+84),TARGET1(t+85),TARGET1(t+86),TARGET1(t+87),TARGET1(t+88),TARGET1(t+89),TARGET1(t+90),TARGET1(t+91),TARGET1(t+92),TARGET1(t+93),TARGET1(t+94),TARGET1(t+95)
336,0.0,0.0,0.0,1.5,69.08,-12.0,0.5,0.0,0.0,1.5,69.06,-12.0,1.0,0.0,0.0,1.6,71.78,-12.0,1.5,0.0,0.0,1.6,71.75,-12.0,2.0,0.0,0.0,1.6,75.20,-12.0,2.5,0.0,0.0,1.5,69.29,-11.0,3.0,0.0,0.0,1.5,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
337,0.5,0.0,0.0,1.5,69.06,-12.0,1.0,0.0,0.0,1.6,71.78,-12.0,1.5,0.0,0.0,1.6,71.75,-12.0,2.0,0.0,0.0,1.6,75.20,-12.0,2.5,0.0,0.0,1.5,69.29,-11.0,3.0,0.0,0.0,1.5,72.56,-11.0,3.5,0.0,0.0,1.4,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
338,1.0,0.0,0.0,1.6,71.78,-12.0,1.5,0.0,0.0,1.6,71.75,-12.0,2.0,0.0,0.0,1.6,75.20,-12.0,2.5,0.0,0.0,1.5,69.29,-11.0,3.0,0.0,0.0,1.5,72.56,-11.0,3.5,0.0,0.0,1.4,72.55,-11.0,4.0,0.0,0.0,1.3,...,0.0,0.0,0.0,0.0,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
339,1.5,0.0,0.0,1.6,71.75,-12.0,2.0,0.0,0.0,1.6,75.20,-12.0,2.5,0.0,0.0,1.5,69.29,-11.0,3.0,0.0,0.0,1.5,72.56,-11.0,3.5,0.0,0.0,1.4,72.55,-11.0,4.0,0.0,0.0,1.3,74.62,-11.0,4.5,0.0,0.0,1.3,...,0.0,0.0,0.0,0.0,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
340,2.0,0.0,0.0,1.6,75.20,-12.0,2.5,0.0,0.0,1.5,69.29,-11.0,3.0,0.0,0.0,1.5,72.56,-11.0,3.5,0.0,0.0,1.4,72.55,-11.0,4.0,0.0,0.0,1.3,74.62,-11.0,4.5,0.0,0.0,1.3,74.61,-11.0,5.0,0.0,0.0,1.3,...,0.0,0.0,0.0,0.0,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52460,22.0,0.0,0.0,2.4,60.10,-4.0,22.5,0.0,0.0,2.5,60.08,-4.0,23.0,0.0,0.0,2.8,59.37,-4.0,23.5,0.0,0.0,2.8,59.35,-4.0,0.0,0.0,0.0,2.9,57.98,-4

In [106]:
# 잘 정렬 되었는지 컬럼을 출력시켜서 확인한다.
# t-1 시점까지 var1 - var6이 잘 들어갔고 그 다음에 t 시점부터 TARGET(발전량)이 들어갔다.
# var6(t-1)까지 변수(train_x)로 학습시키고, 뒤에는 train_y로 학습시키면 되겠다.
reframed.columns[-100:-90]

Index(['var3(t-1)', 'var4(t-1)', 'var5(t-1)', 'var6(t-1)', 'TARGET1(t)',
       'TARGET1(t+1)', 'TARGET1(t+2)', 'TARGET1(t+3)', 'TARGET1(t+4)',
       'TARGET1(t+5)'],
      dtype='object')

In [107]:
# X(풍속, 온도 등)와 y(발전량) 칼럼을 분리해준다.

n_features = len(df_scaled.columns) - 1
n_obs = n_days * n_features
X = reframed.iloc[:, :n_obs]
y = reframed.iloc[:, -future_window:]

In [108]:
y

,TARGET1(t),TARGET1(t+1),TARGET1(t+2),TARGET1(t+3),TARGET1(t+4),TARGET1(t+5),TARGET1(t+6),TARGET1(t+7),TARGET1(t+8),TARGET1(t+9),TARGET1(t+10),TARGET1(t+11),TARGET1(t+12),TARGET1(t+13),TARGET1(t+14),TARGET1(t+15),TARGET1(t+16),TARGET1(t+17),TARGET1(t+18),TARGET1(t+19),TARGET1(t+20),TARGET1(t+21),TARGET1(t+22),TARGET1(t+23),TARGET1(t+24),TARGET1(t+25),TARGET1(t+26),TARGET1(t+27),TARGET1(t+28),TARGET1(t+29),TARGET1(t+30),TARGET1(t+31),TARGET1(t+32),TARGET1(t+33),TARGET1(t+34),TARGET1(t+35),TARGET1(t+36),TARGET1(t+37),TARGET1(t+38),TARGET1(t+39),...,TARGET1(t+56),TARGET1(t+57),TARGET1(t+58),TARGET1(t+59),TARGET1(t+60),TARGET1(t+61),TARGET1(t+62),TARGET1(t+63),TARGET1(t+64),TARGET1(t+65),TARGET1(t+66),TARGET1(t+67),TARGET1(t+68),TARGET1(t+69),TARGET1(t+70),TARGET1(t+71),TARGET1(t+72),TARGET1(t+73),TARGET1(t+74),TARGET1(t+75),TARGET1(t+76),TARGET1(t+77),TARGET1(t+78),TARGET1(t+79),TARGET1(t+80),TARGET1(t+81),TARGET1(t+82),TARGET1(t+83),TARGET1(t+84),TARGET1(t+85),TARGET1(t+86),TARGET1(t+87),TARGET1(t+88),TARGET1(t+89),TARGET1(t+90),TARGET1(t+91),TARGET1(t+92),TARGET1(t+93),TARGET1(t+94),TARGET1(t+95)
336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,7.319893,15.202688,23.085309,30.310866,36.597857,41.759093,45.606058,48.139728,49.265804,48.890445,47.013653,43.823587,39.225395,33.501483,26.651333,19.050491,10.980081,3.660067,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,7.319893,15.202688,23.085309,30.310866,36.597857,41.759093,45.606058,48.139728,49.265804,48.890445,47.013653,43.823587,39.225395,33.501483,26.651333,19.050491,10.980081,3.660067,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,7.319893,15.202688,23.085309,30.310866,36.597857,41.759093,45.606058,48.139728,49.265804,48.890445,47.013653,43.823587,39.225395,33.501483,26.651333,19.050491,10.980081,3.660067,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,7.319893,15.202688,23.085309,30.310866,36.597857,41.759093,45.606058,48.139728,49.265804,48.890445,47.013653,43.823587,39.225395,33.501483,26.651333,19.050491,10.980081,3.660067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.319893,15.202688,23.085309,30.310866,36.597857,41.759093,45.606058,48.139728,49.265804,48.890445,47.013653,43.823587,39.225395,33.501483,26.651333,19.050491,10.980081,3.660067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.34112

In [109]:
# validation을 해야하니까, train으로 준 데이터를 train/test로 자른다.
# 42 = ultimate answer to life the universe and everything

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [110]:
# 잘 잘렸는지 확인한다.
X_train

,var1(t-336),var2(t-336),var3(t-336),var4(t-336),var5(t-336),var6(t-336),var1(t-335),var2(t-335),var3(t-335),var4(t-335),var5(t-335),var6(t-335),var1(t-334),var2(t-334),var3(t-334),var4(t-334),var5(t-334),var6(t-334),var1(t-333),var2(t-333),var3(t-333),var4(t-333),var5(t-333),var6(t-333),var1(t-332),var2(t-332),var3(t-332),var4(t-332),var5(t-332),var6(t-332),var1(t-331),var2(t-331),var3(t-331),var4(t-331),var5(t-331),var6(t-331),var1(t-330),var2(t-330),var3(t-330),var4(t-330),...,var3(t-7),var4(t-7),var5(t-7),var6(t-7),var1(t-6),var2(t-6),var3(t-6),var4(t-6),var5(t-6),var6(t-6),var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var6(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1)
2712,12.0,87.0,1016.0,5.6,36.47,11.0,12.5,125.0,848.0,5.2,36.46,11.0,13.0,113.0,862.0,4.9,37.08,11.0,13.5,84.0,1005.0,4.4,37.08,11.0,14.0,80.0,991.0,4.0,37.86,11.0,14.5,76.0,971.0,3.2,37.85,10.0,15.0,80.0,790.0,2.4,...,0.0,2.0,56.92,-6.0,9.0,77.0,0.0,2.0,58.28,-5.0,9.5,109.0,0.0,1.7,53.97,-4.0,10.0,138.0,2.0,1.5,57.04,-4.0,10.5,132.0,1.0,1.4,52.85,-3.0,11.0,227.0,19.0,1.4,55.55,-3.0,11.5,299.0,55.0,1.7,55.53,-3.0
15176,4.0,0.0,0.0,0.7,66.90,-1.0,4.5,0.0,0.0,0.7,66.92,-1.0,5.0,0.0,0.0,0.8,70.74,-2.0,5.5,0.0,0.0,0.8,70.75,-2.0,6.0,0.0,0.0,0.8,70.03,-2.0,6.5,0.0,0.0,0.9,65.01,-1.0,7.0,24.0,296.0,1.1,...,0.0,2.5,67.39,-4.0,1.0,0.0,0.0,2.6,64.10,-4.0,1.5,0.0,0.0,2.6,64.08,-4.0,2.0,0.0,0.0,2.7,61.53,-4.0,2.5,0.0,0.0,2.6,61.52,-4.0,3.0,0.0,0.0,2.6,59.71,-4.0,3.5,0.0,0.0,2.5,59.71,-4.0
45061,18.5,20.0,0.0,3.4,86.30,18.0,19.0,5.0,0.0,2.8,87.84,18.0,19.5,0.0,0.0,2.4,87.86,18.0,20.0,0.0,0.0,2.0,88.54,18.0,20.5,0.0,0.0,1.8,88.53,17.0,21.0,0.0,0.0,1.7,89.30,17.0,21.5,0.0,0.0,1.4,...,20.0,2.4,36.89,26.0,15.5,203.0,13.0,2.3,39.16,25.0,16.0,273.0,111.0,2.3,43.44,25.0,16.5,181.0,20.0,2.0,46.17,24.0,17.0,64.0,0.0,1.8,52.54,23.0,17.5,47.0,0.0,1.6,55.88,22.0,18.0,30.0,0.0,1.5,61.57,21.0
47122,17.0,90.0,463.0,2.0,30.99,25.0,17.5,64.0,333.0,1.6,35.00,22.0,18.0,30.0,151.0,1.3,45.30,20.0,18.5,0.0,0.0,1.4,48.23,19.0,19.0,0.0,0.0,1.5,50.18,19.0,19.5,0.0,0.0,1.6,53.47,18.0,20.0,0.0,0.0,1.8,...,9.0,2.5,21.24,29.0,14.0,286.0,494.0,2.5,21.62,29.0,14.5,257.0,462.0,2.3,22.93,28.0,15.0,278.0,236.0,2.2,23.71,27.0,15.5,226.0,281.0,1.9,25.16,26.0,16.0,196.0,223.0,1.7,28.47,26.0,16.5,149.0,198.0,1.1,32.13,24.0
17469,22.5,0.0,0.0,4.5,47.03,-8.0,23.0,0.0,0.0,4.5,46.25,-9.0,23.5,0.0,0.0,4.4,46.27,-9.0,0.0,0.0,0.0,4.4,47.26,-9.0,0.5,0.0,0.0,4.3,47.28,-9.0,1.0,0.0,0.0,4.2,52.17,-9.0,1.5,0.0,0.0,3.9,...,0.0,0.5,71.94,-10.0,19.5,0.0,0.0,0.5,71.94,-10.0,20.0,0.0,0.0,0.5,74.06,-11.0,20.5,0.0,0.0,0.6,74.07,-11.0,21.0,0.0,0.0,0.7,70.51,-12.0,21.5,0.0,0.0,0.7,70.52,-12.0,22.0,0.0,0.0,0.8,72.42,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11620,2.0,0.0,0.0,1.8,49.47,12.0,2.5,0.0,0.0,1.7,49.46,12.0,3.0,0.0,0.0,1.7,47.39,12.0,3.5,0.0,0.0,1.6,47.39,11.0,4.0,0.0,0.0,1.6,47.81,11.0,4.5,0.0,0.0,1.5,47.82,11.0,5.0,0.0,0.0,1.4,...,0.0,1.5,57.31,18.0,23.0,0.0,0.0,1.6,56.76,18.0,23.5,0.0,0.0,1.7,60.51,17.0,0.0,0.0,0.0,1.8,60.15,17.0,0.5,0.0,0.0,2.0,60.15,17.0,1.0,0.0,0.0,2.2,60.31,17.0,1.5,0.0,0.0,2.4,60.32,17.0
45068,22.0,0.0,0.0,1.2,96.33,17.0,22.5,0.0,0.0,0.8,96.33,17.0,23.0,0.0,0.0,0.4,96.45,17.0,23.5,0.0,0.0,0.7,96.46,17.0,0.0,0.0,0.0,1.1,95.66,17.0,0.5,0.0,0.0,1.1,95.65,17.0,1.0,0.0,0.0,1.1,...,0.0,1.4,65.51,20.0,19.0,3.0,0.0,1.3,72.04,19.0,19.5,0.0,0.0,1.4,76.72,19.0,20.0,0.0,0.0,1.5,75.84,19.0,20.5,0.0,0.0,1.9,80.82,18.0,21.0,0.0,0.0,2.3,79.23,18.0,21.5,0.0,0.0,2.5,79.24,17.0
38494,23.0,0.0,0.0,4.1,43.97,-7.0,23.5,0.0,0.0,4.1,

In [111]:
trainset = X_train.values
trainset = trainset.reshape(trainset.shape[0],-1,6)
trainset_y = y_train.values
trainset_y = trainset_y.reshape(trainset_y.shape[0],96,1)

testset = X_test.values
testset = testset.reshape(testset.shape[0],-1,6)
testset_y = y_test.values
testset_y = testset_y.reshape(testset_y.shape[0],96,1)


train_X, train_y, test_X, test_y = trainset, trainset_y, testset, testset_y
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(36490, 336, 6) (36490, 96, 1) (15639, 336, 6) (15639, 96, 1)


In [112]:
# x 데이터와 y 데이터를 합친다.

train_data = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
val_data = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))
train_data

In [113]:
# 앞에서 합친 데이터를 원하는 batch_size로 잘라서 학습준비
batches = 100

train_set = DataLoader(train_data, batch_size = batches, shuffle=True)
test_set = DataLoader(val_data, batch_size = batches, shuffle=True)

In [114]:
k = 0
for data in train_set:
    if k < 2:
      x, _ = data
      print(x)
    else:
      break
    k +=1

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 1.6000e+00, 8.0370e+01,
          8.0000e+00],
         [1.5000e+00, 0.0000e+00, 0.0000e+00, 1.7000e+00, 8.0370e+01,
          7.0000e+00],
         [2.0000e+00, 0.0000e+00, 0.0000e+00, 1.8000e+00, 7.6430e+01,
          7.0000e+00],
         ...,
         [2.3500e+01, 0.0000e+00, 0.0000e+00, 2.2000e+00, 6.1320e+01,
          9.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 2.4000e+00, 5.7560e+01,
          9.0000e+00],
         [5.0000e-01, 0.0000e+00, 0.0000e+00, 2.5000e+00, 5.7560e+01,
          9.0000e+00]],

        [[2.3500e+01, 0.0000e+00, 0.0000e+00, 3.7000e+00, 7.8590e+01,
          1.7000e+01],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 3.6000e+00, 7.5530e+01,
          1.7000e+01],
         [5.0000e-01, 0.0000e+00, 0.0000e+00, 3.5000e+00, 7.5510e+01,
          1.6000e+01],
         ...,
         [2.2000e+01, 0.0000e+00, 0.0000e+00, 1.7000e+00, 9.7390e+01,
          1.5000e+01],
         [2.2500e+01, 0.0000e+00, 0.0000

In [115]:
# 하나씩 꺼내서 확인하기
# size 확인.. 나중에 학습에 써먹기 위함.
dateiter = iter(train_set)
tx,ty = dateiter.next()
print(tx.size())
print(ty.size())

torch.Size([100, 336, 6])
torch.Size([100, 96, 1])


In [116]:
###### 이 밑은 해보면서 공부중......... #####

In [117]:
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(Model, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building LSTM
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0,c0))

        # Index hidden state of last time step
        out = self.fc(out[:, -1, :]) 

        return out 

In [118]:
input_dim = 6
hidden_dim = 100
output_dim = 96
layer_dim = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [125]:
model = Model(input_dim, hidden_dim, layer_dim, output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [128]:
def tilted_loss(q,y,f):
    e = (y-f)
    return torch.mean(torch.max(q*e, (q-1)*e), axis=-1)

In [120]:
print(model)

Model(
  (lstm): LSTM(6, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=96, bias=True)
)


In [121]:
# input shape랑... 왜 안들어가지는지 모르겠다.
# loss function (quantile) 만 해결하면 된다....!

In [133]:
epochs = 6
k = 0
quantile = [0.5]
for epoch in range(epochs):
    for i, (train, train_target) in enumerate(train_set):

#        if(torch.cuda.is_available()):
#
#          model.cuda()# rnn is your model 
#          train = train.cuda() 
#          train_target = train_target.cuda()


        # Load data as a torch tensor with gradient accumulation abilities
        train = train.float()
        train_target = train_target.float()
        train = train.requires_grad_()
        train_target = train_target

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(train.float())

        # Calculate Loss: softmax --> cross entropy loss
        train_target = train_target.squeeze_()
        # loss = criterion(outputs, train_target)
        loss = tilted_loss(quantile[0], outputs, train_target)

        # Getting gradients w.r.t. parameters
        loss.mean().backward()

        # Updating parameters
        optimizer.step()
    k+=1  
    # print('Epoch: {}. Loss: {}. '.format(k, np.around(loss.item(), 4)))
    print('Epoch: {}'.format(k))

Epoch: 1


KeyboardInterrupt: ignored